# SP500 Attempt to predict index using other indicators with XGBOOST 

*source:* https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/

In [12]:
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime as dt
import pyodbc
import datetime
import plotly.express as px
import sqlite3
import os
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [13]:
#Destination folder (comment unused directory)
#1 
#os.chdir("C:/Users/Lukasz/Documents/GitHub/Analysis-and-forecast-of-stock-exchange-price-based-on-selected-indicators-using-machine-learning/data")
#2
os.chdir("F:/GitHub/Analysis-and-forecast-of-stock-exchange-price-based-on-selected-indicators-using-machine-learning/data")

# Create your connection.
conn = sqlite3.connect('predictions.db')
c = conn.cursor()

In [14]:
indicator="SP500"
#Get SQLLite select statement for SP500 index and indicators
df = pd.read_sql_query("""select x."Date", c19."Index" Covid19, co."Index" as Crude_Oil, 
cr."Index" as Copper, s."Index" as Silver, p."Index" as Platinum,
ip."Index" as Industrial_Production, pa."Index" as Palladium, x."Index" as """+indicator+"""
 from """+indicator+""" as x
  LEFT OUTER JOIN Covid19 as c19 ON x.Date = C19.Date
  LEFT OUTER JOIN Crude_Oil as co ON x.Date=co.Date
  LEFT OUTER JOIN Copper as cr ON x.Date=cr.Date
  LEFT OUTER JOIN Silver as s ON x.Date=s.Date
  LEFT OUTER JOIN Platinum as p ON x.Date=p.Date
  LEFT OUTER JOIN Industrial_Prod as ip ON strftime('%Y-%m', x.Date)=strftime('%Y-%m', ip.Date)
  LEFT OUTER JOIN Palladium as pa ON x.Date=pa.Date
Where x."Index" and co."Index" and cr."Index" and s."Index" and p."Index"
and ip."Index" and pa."Index" IS NOT NULL
and x.Date < '2020-04-01'
""", conn)

#Delete date column
df = df.drop(columns=['Date'])

#Round all columns
#df = df.round(1)

df.tail()

,Covid19,Crude_Oil,Copper,Silver,Platinum,Industrial_Production,Palladium,SP500
2510,416881.0,24.49,2.2040,13.965,745.50,103.664,2247.6,2475.56
2511,468092.0,22.60,2.1780,14.415,737.10,103.664,2226.1,2630.07
2512,527839.0,21.51,2.1720,14.315,740.82,103.664,2196.8,2541.47
2513,715377.0,20.09,2.1555,14.055,723.84,103.664,2197.6,2626.65
2514,777187.0,20.48,2.2280,13.930,728.80,103.664,2304.8,2584.59


In [15]:
df=df.fillna(0)
df.tail()

,Covid19,Crude_Oil,Copper,Silver,Platinum,Industrial_Production,Palladium,SP500
2510,416881.0,24.49,2.2040,13.965,745.50,103.664,2247.6,2475.56
2511,468092.0,22.60,2.1780,14.415,737.10,103.664,2226.1,2630.07
2512,527839.0,21.51,2.1720,14.315,740.82,103.664,2196.8,2541.47
2513,715377.0,20.09,2.1555,14.055,723.84,103.664,2197.6,2626.65
2514,777187.0,20.48,2.2280,13.930,728.80,103.664,2304.8,2584.59


In [16]:
df.to_csv(r'F:/GitHub/Analysis-and-forecast-of-stock-exchange-price-based-on-selected-indicators-using-machine-learning/data\xgboost.csv',index=False,header=False)

In [17]:
dataset = loadtxt(r'F:/GitHub/Analysis-and-forecast-of-stock-exchange-price-based-on-selected-indicators-using-machine-learning/data\xgboost.csv', delimiter=",")
# split data into X and y
X = dataset[:,0:7]
Y = dataset[:,7]
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = r2_score(y_test, predictions)
print("R2_score: %.2f%%" % (accuracy * 100.0))

R2_score: 94.13%
